<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Cleaning-and-Feature-Engineering" data-toc-modified-id="Data-Cleaning-and-Feature-Engineering-1">Data Cleaning and Feature Engineering</a></span></li><li><span><a href="#Podcast-Date-/-Time-Series-Features" data-toc-modified-id="Podcast-Date-/-Time-Series-Features-2">Podcast Date / Time Series Features</a></span><ul class="toc-item"><li><span><a href="#Social-Feed-/-External-Site-Classifications" data-toc-modified-id="Social-Feed-/-External-Site-Classifications-2.1">Social Feed / External Site Classifications</a></span></li></ul></li></ul></div>

In [1]:
import pickle
import random
import sys
from datetime import datetime
import pandas as pd
import math
import time
import re

import os


In [2]:
with open('../scraped/channel/by_category/Arts.pickle','rb') as file:
    df = pickle.load(file)

df.head(2)

,title,chan_url,num_comments,author,isExplicit,sub_count,play_count,ch_feed-socials,ep_total,recent_eps,hover_text_concat,chan_desc,cover_img_url,first_release,description
0,Fresh Air,https://castbox.fm/channel/Fresh-Air-id431951,126,NPR,0,167880,4705927,"[https://twitter.com/nprfreshair, https://www....",50,"[[2019-10-03, 00:48:44, 0], [2019-10-02, 00:49...",President Trump made building a border wall be...,"Fresh Air from WHYY, the Peabody Award-winning...",https://is3-ssl.mzstatic.com/image/thumb/Podca...,2019-08-07,NaN
1,The Moth,https://castbox.fm/channel/The-Moth-id12,129,The Moth,0,143346,2419216,"[https://twitter.com/TheMoth, https://www.face...",157,"[[2019-10-01, 00:51:10, 8], [2019-09-24, 00:51...","This week, Two women meet by chance on a dark ...","Since its launch in 1997, The Moth has present...",https://is5-ssl.mzstatic.com/image/thumb/Podca...,2017-03-28,NaN


## Data Cleaning and Feature Engineering

## Podcast Date / Time Series Features

Use the `recent_eps` column, along with the `first_release` column to build features that assess recency, frequency, and age of podcast.

We'll build feature functions from a single podcast as a sample.

In [3]:
df.iloc[0]

title                                                        Fresh Air
chan_url                 https://castbox.fm/channel/Fresh-Air-id431951
num_comments                                                       126
author                                                             NPR
isExplicit                                                           0
sub_count                                                       167880
play_count                                                     4705927
ch_feed-socials      [https://twitter.com/nprfreshair, https://www....
ep_total                                                            50
recent_eps           [[2019-10-03, 00:48:44, 0], [2019-10-02, 00:49...
hover_text_concat    President Trump made building a border wall be...
chan_desc            Fresh Air from WHYY, the Peabody Award-winning...
cover_img_url        https://is3-ssl.mzstatic.com/image/thumb/Podca...
first_release                                               2019-08-07
descri

In [4]:
sample_eps = df.iloc[0].recent_eps
sample_eps

[['2019-10-03', '00:48:44', 0],
 ['2019-10-02', '00:49:21', 6],
 ['2019-10-01', '00:49:13', 4],
 ['2019-09-30', '00:50:06', 4],
 ['2019-09-27', '00:50:24', 2],
 ['2019-09-27', '00:48:28', 1],
 ['2019-09-26', '00:48:59', 9],
 ['2019-09-25', '00:49:41', 2],
 ['2019-09-24', '00:49:44', 11],
 ['2019-09-23', '00:49:55', 4]]

In [29]:
# Let's build a column called 'recent_ep_rate'
# which calculates the average days b/t episode releases
# for the last 10 or so episodes.

def convert_ep_date(string):
    return datetime.strptime(string, '%Y-%m-%d')

def recent_ep_mean_dist(recent_eps):
    ep_list = [convert_ep_date(ep[0]) for ep in recent_eps]
#     print(ep_list)
    
    days_bt_eps = []
    for i in range(len(ep_list)-1):
        days_bt_eps += [abs(ep_list[i+1] - ep_list[i])]

    return pd.Series(days_bt_eps).dt.days.mean()
    

recent_ep_mean_dist(sample_eps)

1.1111111111111112

Load feature column into dataframe.

In [6]:
ep_cols = ['title', 'play_count', 'ep_total','first_release','recent_eps','recent_ep_spacing']
df['recent_ep_spacing'] = df.recent_eps.apply(recent_ep_mean_dist)
df[ep_cols].head(3)

,title,play_count,ep_total,first_release,recent_eps,recent_ep_spacing
0,Fresh Air,4705927,50,2019-08-07,"[[2019-10-03, 00:48:44, 0], [2019-10-02, 00:49...",1.111111
1,The Moth,2419216,157,2017-03-28,"[[2019-10-01, 00:51:10, 8], [2019-09-24, 00:51...",4.666667
2,TED Talks Daily,27571564,975,2016-02-08,"[[2019-10-03, 00:13:43, 3], [2019-10-02, 00:16...",1.111111


Let's build a column called `lifetime_ep_freq` which takes first and last episode date, and divides that distance by total.

In [14]:
test = convert_ep_date('2019-10-03')

In [15]:
test_2 = convert_ep_date('2019-11-03')

In [17]:
delta = (test_2 - test)

In [19]:
delta.total_seconds()

2678400.0

In [28]:
ep_cols_2 = ep_cols + ['lifetime_ep_freq']


def lifetime_ep_freq(row):
    '''
    
    approx. episodes / day
    
    =  [ep count] / [age of podcast (up to last activity) in days]
    
    Example:
    df['lifetime_ep_freq'] = df.apply(lifetime_ep_freq, axis=1)
    
    '''
    
    last_ep_date = convert_ep_date(row['recent_eps'][0][0])
    release_date = convert_ep_date(row['first_release'])
    ep_total = row['ep_total']
    
    
    
    age = (last_ep_date - release_date).total_seconds() / (24. * 60. * 60.)
    
    if age == 0:
        freq = 0
    else:
        freq = ep_total / age

    print(last_ep_date, release_date, ep_total, age, freq)

        
    return freq
    
    
df['lifetime_ep_freq'] = df.apply(lifetime_ep_freq, axis=1)
df[ep_cols_2].head(3)

2019-10-03 00:00:00 2019-08-07 00:00:00 50 57.0 0.8771929824561403
2019-10-01 00:00:00 2017-03-28 00:00:00 157 917.0 0.17121046892039257
2019-10-03 00:00:00 2016-02-08 00:00:00 975 1333.0 0.7314328582145536
2019-10-01 00:00:00 2010-09-23 00:00:00 404 3295.0 0.12261001517450683
2019-09-19 00:00:00 2017-09-01 00:00:00 36 748.0 0.0481283422459893
2019-10-01 00:00:00 2015-04-27 00:00:00 224 1618.0 0.138442521631644
2019-08-15 00:00:00 2019-05-03 00:00:00 9 104.0 0.08653846153846154
2019-09-26 00:00:00 2015-10-16 00:00:00 248 1441.0 0.17210270645385148
2019-09-25 00:00:00 2019-07-09 00:00:00 11 78.0 0.14102564102564102
2019-10-01 00:00:00 2017-06-02 00:00:00 75 851.0 0.0881316098707403
2019-10-03 00:00:00 2017-12-13 00:00:00 86 659.0 0.13050075872534142
2019-10-01 00:00:00 2018-09-24 00:00:00 58 372.0 0.15591397849462366
2019-10-03 00:00:00 2019-10-03 00:00:00 176 0.0 0
2019-09-30 00:00:00 2019-08-19 00:00:00 7 42.0 0.16666666666666666
2019-09-27 00:00:00 2019-09-27 00:00:00 262 0.0 0
2019-

,title,play_count,ep_total,first_release,recent_eps,recent_ep_spacing,lifetime_ep_freq
0,Fresh Air,4705927,50,2019-08-07,"[[2019-10-03, 00:48:44, 0], [2019-10-02, 00:49...",1.111111,0.877193
1,The Moth,2419216,157,2017-03-28,"[[2019-10-01, 00:51:10, 8], [2019-09-24, 00:51...",4.666667,0.171210
2,TED Talks Daily,27571564,975,2016-02-08,"[[2019-10-03, 00:13:43, 3], [2019-10-02, 00:16...",1.111111,0.731433


Let's calculate average episode length for the `recent_eps` and save as `avg_ep_len`.

In [83]:
ep_cols_3 = ep_cols_2 + ['avg_ep_len']

def avg_ep_len(recent_eps):
    '''
    
    Average length of recent 10 or so episodes.
    
    '''
    ep_lens = [time.strptime(ep[1], '%H:%M:%S') for ep in recent_eps]
    
#     print(ep_lens)
    avg_ep_len = np.mean(ep_lens)

    return avg_ep_len
    
    
df['avg_ep_len'] = df.recent_eps.apply(avg_ep_len) 
df[ep_cols_3].head(3)

,title,play_count,ep_total,first_release,recent_eps,recent_ep_spacing,lifetime_ep_freq,avg_ep_len
0,Fresh Air,4705927,50,2019-08-07,"[[2019-10-03, 00:48:44, 0], [2019-10-02, 00:49...",1.111111,1.140000,220.488889
1,The Moth,2419216,157,2017-03-28,"[[2019-10-01, 00:51:10, 8], [2019-09-24, 00:51...",4.666667,5.840764,219.144444
2,TED Talks Daily,27571564,975,2016-02-08,"[[2019-10-03, 00:13:43, 3], [2019-10-02, 00:16...",1.111111,1.367179,216.344444


### Social Feed / External Site Classifications

Create feature columns to describe quantity of social/external feeds.

Additional scraping of each social feed to assess activity of feed will follow.

In [179]:
social_cols = ['title','play_count', 'ch_feed-socials']
df[social_cols].head(3)

,title,play_count,ch_feed-socials
0,Fresh Air,4705927,"[https://twitter.com/nprfreshair, https://www...."
1,The Moth,2419216,"[https://twitter.com/TheMoth, https://www.face..."
2,TED Talks Daily,27571564,"[https://twitter.com/TEDTalks, https://www.fac..."


In [98]:
# First, let's print a bunch of these feeds, so we know the overall format.
for label, content in df['ch_feed-socials'].sample(10).items():
    print('content:', content, sep='\n')

content:
['https://twitter.com/snapjudgment', 'https://www.facebook.com/SnapJudgment', 'https://www.wnycstudios.org/podcasts/snapjudgment']
content:
['https://twitter.com/LaurenConrad', 'https://www.facebook.com/laurenconrad', 'https://laurenconrad.com/']
content:
['https://twitter.com/TEDTalks', 'https://www.facebook.com/TED', 'https://www.instagram.com/ted', 'https://www.youtube.com/TEDtalksDirector', 'https://www.ted.com/talks']
content:
[]
content:
['https://twitter.com/RISKshow', 'https://www.facebook.com/riskshow', 'http://www.risk-show.com']
content:
['https://twitter.com/TheMoth', 'https://www.facebook.com/TheMoth', 'https://www.instagram.com/mothstories', 'https://www.youtube.com/mothstories', 'http://themoth.org/']
content:
['https://twitter.com/MythPodcast', 'https://www.facebook.com/mythpodcast', 'https://www.instagram.com/mythpodcast', 'https://www.youtube.com/channel/UCM54A0R5z0BwMpEWVYtqbkQ', 'http://www.mythpodcast.com']
content:
['https://twitter.com/TraderJoesList', '

In [160]:
def has_domain(row, social_domain):
#     print(row)
    ch_feed_socials = row
    for link in ch_feed_socials:
        pattern = '.*' + social_domain + '.*'
#         print(pattern, link)
#         print(re.match(pattern, link))
        if re.match(pattern, link):
            return 1
    return 0

In [188]:
social_domain = 'twitter'
df[social_domain] = df['ch_feed-socials'][:10].apply(has_domain,social_domain=social_domain)

social_domains = ['twitter', 'facebook', 'youtube', 'instagram']

for domain in social_domains:
    df[domain] = df['ch_feed-socials'].apply(has_domain,social_domain=domain)

In [189]:
df

,title,chan_url,num_comments,author,isExplicit,sub_count,play_count,ch_feed-socials,ep_total,recent_eps,...,first_release,description,recent_ep_rate,lifetime_ep_freq,avg_ep_len,recent_ep_spacing,twitter,facebook,youtube,instagram
0,Fresh Air,https://castbox.fm/channel/Fresh-Air-id431951,126,NPR,0,167880,4705927,"[https://twitter.com/nprfreshair, https://www....",50,"[[2019-10-03, 00:48:44, 0], [2019-10-02, 00:49...",...,2019-08-07,NaN,1.111111,1.140000,220.488889,1.111111,1,1,1,1
1,The Moth,https://castbox.fm/channel/The-Moth-id12,129,The Moth,0,143346,2419216,"[https://twitter.com/TheMoth, https://www.face...",157,"[[2019-10-01, 00:51:10, 8], [2019-09-24, 00:51...",...,2017-03-28,NaN,4.666667,5.840764,219.144444,4.666667,1,1,1,1
2,TED Talks Daily,https://castbox.fm/channel/TED-Talks-Daily-id4541,710,TED,0,1382334,27571564,"[https://twitter.com/TEDTalks, https://www.fac...",975,"[[2019-10-03, 00:13:43, 3], [2019-10-02, 00:16...",...,2016-02-08,NaN,1.111111,1.367179,216.344444,1.111111,1,1,1,1
3,99% Invisible,https://castbox.fm/channel/id18,208,Roman Mars,0,177413,3149718,"[https://twitter.com/romanmars, http://99perce...",404,"[[2019-10-01, 00:33:53, 11], [2019-09-24, 00:4...",...,2010-09-23,NaN,7.000000,8.155941,217.844444,7.000000,1,0,0,0
4,Snap Judgment Presents: Spooked,https://castbox.fm/channel/Snap-Judgment-Prese...,208,Snap Judgment and WNYC Studios,0,23327,322310,"[https://twitter.com/SpookedPod, https://www.f...",36,"[[2019-09-19, 00:21:11, 7], [2019-09-13, 00:30...",...,2017-09-01,NaN,37.444444,20.777778,218.022222,37.444444,1,1,0,0
5,Myths and Legends,https://castbox.fm/channel/Myths-and-Legends-i...,327,Jason Weiser Carissa Weiser / Bardic,0,152126,2568707,"[https://twitter.com/MythPodcast, https://www....",224,"[[2019-10-01, 00:39:57, 4], [2019-09-24, 00:40...",...,2015-04-27,NaN,7.000000,7.223214,219.344444,7.000000,1,1,1,1
6,Nice Try!,https://castbox.fm/channel/Nice-Try!-id2112178,6,Curbed,0,17593,58893,"[https://twitter.com/curbed, https://www.faceb...",9,"[[2019-08-15, 00:46:51, 5], [2019-07-18, 00:36...",...,2019-05-03,NaN,13.000000,11.555556,219.000000,13.000000,1,1,0,1
7,Snap Judgment,https://castbox.fm/channel/Snap-Judgment-id131,127,Snap Judgment and WNYC Studios,0,43501,829582,"[https://twitter.com/snapjudgment, https://www...",248,"[[2019-09-26, 00:28:37, 8], [2019-09-19, 00:36...",...,2015-10-16,NaN,5.444444,5.810484,217.400000,5.444444,1,1,0,0
8,The Beauty Closet,https://castbox.fm/channel/The-Beauty-Closet-i...,1,Goop Inc and Cadence 13,0,354,4732,[https://goop.com/thepodcast],11,"[[2019-09-25, 00:43:28, 1], [2019-09-18, 00:50...",...,2019-07-09,NaN,7.000000,7.090909,219.166667,7.000000,0,0,0,0
9,LeVar Burton Reads,https://castbox.fm/channel/LeVar-Burton-Reads-...,153,LeVar Burton and Stitcher,1,29068,493187,"[https://twitter.com/levarburton, https://www....",75,"[[2019-10-01, 00:52:15, 5], [2019-09-24, 00:39...",...,2017-06-02,NaN,9.444444,11.346667,219.755556,9.444444,1,1,0,0


In [184]:
all_social_cols = social_cols.extend(social_domains)
df[all_social_cols].head(10)

KeyError: None

In [170]:
re.match('.*twitter.*','https://twitter.com/nprfreshair').group()

'https://twitter.com/nprfreshair'

In [180]:
social_cols.extend(social_domains)

In [181]:
social_cols

['title',
 'play_count',
 'ch_feed-socials',
 'twitter',
 'facebook',
 'youtube',
 'instagram']

In [190]:
df

,title,chan_url,num_comments,author,isExplicit,sub_count,play_count,ch_feed-socials,ep_total,recent_eps,...,first_release,description,recent_ep_rate,lifetime_ep_freq,avg_ep_len,recent_ep_spacing,twitter,facebook,youtube,instagram
0,Fresh Air,https://castbox.fm/channel/Fresh-Air-id431951,126,NPR,0,167880,4705927,"[https://twitter.com/nprfreshair, https://www....",50,"[[2019-10-03, 00:48:44, 0], [2019-10-02, 00:49...",...,2019-08-07,NaN,1.111111,1.140000,220.488889,1.111111,1,1,1,1
1,The Moth,https://castbox.fm/channel/The-Moth-id12,129,The Moth,0,143346,2419216,"[https://twitter.com/TheMoth, https://www.face...",157,"[[2019-10-01, 00:51:10, 8], [2019-09-24, 00:51...",...,2017-03-28,NaN,4.666667,5.840764,219.144444,4.666667,1,1,1,1
2,TED Talks Daily,https://castbox.fm/channel/TED-Talks-Daily-id4541,710,TED,0,1382334,27571564,"[https://twitter.com/TEDTalks, https://www.fac...",975,"[[2019-10-03, 00:13:43, 3], [2019-10-02, 00:16...",...,2016-02-08,NaN,1.111111,1.367179,216.344444,1.111111,1,1,1,1
3,99% Invisible,https://castbox.fm/channel/id18,208,Roman Mars,0,177413,3149718,"[https://twitter.com/romanmars, http://99perce...",404,"[[2019-10-01, 00:33:53, 11], [2019-09-24, 00:4...",...,2010-09-23,NaN,7.000000,8.155941,217.844444,7.000000,1,0,0,0
4,Snap Judgment Presents: Spooked,https://castbox.fm/channel/Snap-Judgment-Prese...,208,Snap Judgment and WNYC Studios,0,23327,322310,"[https://twitter.com/SpookedPod, https://www.f...",36,"[[2019-09-19, 00:21:11, 7], [2019-09-13, 00:30...",...,2017-09-01,NaN,37.444444,20.777778,218.022222,37.444444,1,1,0,0
5,Myths and Legends,https://castbox.fm/channel/Myths-and-Legends-i...,327,Jason Weiser Carissa Weiser / Bardic,0,152126,2568707,"[https://twitter.com/MythPodcast, https://www....",224,"[[2019-10-01, 00:39:57, 4], [2019-09-24, 00:40...",...,2015-04-27,NaN,7.000000,7.223214,219.344444,7.000000,1,1,1,1
6,Nice Try!,https://castbox.fm/channel/Nice-Try!-id2112178,6,Curbed,0,17593,58893,"[https://twitter.com/curbed, https://www.faceb...",9,"[[2019-08-15, 00:46:51, 5], [2019-07-18, 00:36...",...,2019-05-03,NaN,13.000000,11.555556,219.000000,13.000000,1,1,0,1
7,Snap Judgment,https://castbox.fm/channel/Snap-Judgment-id131,127,Snap Judgment and WNYC Studios,0,43501,829582,"[https://twitter.com/snapjudgment, https://www...",248,"[[2019-09-26, 00:28:37, 8], [2019-09-19, 00:36...",...,2015-10-16,NaN,5.444444,5.810484,217.400000,5.444444,1,1,0,0
8,The Beauty Closet,https://castbox.fm/channel/The-Beauty-Closet-i...,1,Goop Inc and Cadence 13,0,354,4732,[https://goop.com/thepodcast],11,"[[2019-09-25, 00:43:28, 1], [2019-09-18, 00:50...",...,2019-07-09,NaN,7.000000,7.090909,219.166667,7.000000,0,0,0,0
9,LeVar Burton Reads,https://castbox.fm/channel/LeVar-Burton-Reads-...,153,LeVar Burton and Stitcher,1,29068,493187,"[https://twitter.com/levarburton, https://www....",75,"[[2019-10-01, 00:52:15, 5], [2019-09-24, 00:39...",...,2017-06-02,NaN,9.444444,11.346667,219.755556,9.444444,1,1,0,0


In [30]:
with open('../scraped/channel/podcast_chan_dict-corrected-urls-v2.pickle', 'rb') as file:
    chan_dict = pickle.load(file)

In [32]:
len(chan_dict.keys())

116